# Problem Statement 

You want to collect text data using Twitter API

https://developer.twitter.com/en


__Consumer key__

__API KEY:__  8Xu9vfB3mvDoDBcbdpE5U4se5

__API Secret key:__ 1XMBfMhWqO21vnqQR2LjwizbLd3mHxPHKfWpJTQzVPJjRgGnPE

__Access token:__ 374514313-RpCn2ZBKQUPyJldXgKk6uK0I2Aeu0nZwOc5vfIfQ

__Access token secret:__ Ysagnu8gYUwnah2q69PbyPTXBgEnkltjxeD9r4gdEetXo

In [24]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

In [25]:
import tweepy
import openpyxl
from tweepy import OAuthHandler

In [26]:
# Credentials

consumer_key = "8Xu9vfB3mvDoDBcbdpE5U4se5"
consumer_secret = "1XMBfMhWqO21vnqQR2LjwizbLd3mHxPHKfWpJTQzVPJjRgGnPE"
access_token = "374514313-RpCn2ZBKQUPyJldXgKk6uK0I2Aeu0nZwOc5vfIfQ"
access_token_secret = "Ysagnu8gYUwnah2q69PbyPTXBgEnkltjxeD9r4gdEetXo"

In [27]:
# CALL API

auth =  tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [28]:
#Lets provide the query which I need to search 

query = "Covid Vaccine India"

#Fetch Tweets

Tweets = api.search(query,count=10,lang='en',exclude='retweets', tweet_mode='extended')

for tweet in Tweets:
    print(tweet)
    print("==============================")

Status(_api=<tweepy.api.API object at 0x0000021C1B2C9988>, _json={'created_at': 'Sat Mar 06 19:30:32 +0000 2021', 'id': 1368282869336997890, 'id_str': '1368282869336997890', 'full_text': '#Bollywood #HemaMalini #vaccination #SaifAliKhan #KamalHaasan PHOTOS: Hema Malini receives COVID-19 vaccine shot in Mumbai - Times of India https://t.co/yETeblwsu0', 'truncated': False, 'display_text_range': [0, 163], 'entities': {'hashtags': [{'text': 'Bollywood', 'indices': [0, 10]}, {'text': 'HemaMalini', 'indices': [11, 22]}, {'text': 'vaccination', 'indices': [23, 35]}, {'text': 'SaifAliKhan', 'indices': [36, 48]}, {'text': 'KamalHaasan', 'indices': [49, 61]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/yETeblwsu0', 'expanded_url': 'http://dlvr.it/Rv68wd', 'display_url': 'dlvr.it/Rv68wd', 'indices': [140, 163]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://dlvrit.com/" rel="nofollow">dlvr.it</a>', 'in_reply_to_status_id': 

# Getting the Tweets and some of its related Attributes 

In [29]:
df = pd.DataFrame(columns=[
    'Tweets','User','User_statuses_count',
    'user_followers','User_location','User_verified',
    'fav_count','rt_count','tweet_data'
])
df

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_data


In [30]:
def stream(data,file_name):
    i=0
    for tweet in tweepy.Cursor(api.search,q=data,count=100,lang='en').items():
        print(i,end='\r')
        df.loc[i,'Tweets'] = tweet.text
        df.loc[i,'User'] = tweet.user.name
        df.loc[i,'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i,'user_followers'] = tweet.user.followers_count
        df.loc[i,'User_location'] = tweet.user.location
        df.loc[i,'User_verified'] = tweet.user.verified
        df.loc[i,'fav_count'] = tweet.favorite_count
        df.loc[i,'rt_count'] = tweet.retweet_count
        df.loc[i,'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i=i+1
        if i == 1000:
            break
        else:
            pass

In [31]:
stream(data=["Covid Vaccine India"], file_name='my_tweets')

In [32]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_data,tweet_date
0,RT @DalaiLama: His Holiness the Dalai Lama rec...,Boreas the North Wind,13559,367,Here and there,False,0,1788,NaN,2021-03-06 19:31:02
1,RT @DalaiLama: His Holiness the Dalai Lama rec...,Judith Weiler,4189,890,"Stow, Massachusetts, USA",False,0,1788,NaN,2021-03-06 19:30:55
2,RT @AabidMagami: Breaking :\n\nGujarat Man Inf...,Yash |🇮🇳,21574,650,India,False,0,137,NaN,2021-03-06 19:30:52
3,#Bollywood #HemaMalini #vaccination #SaifAliKh...,FlaShBloG Live Entertainment ©,642616,1045,Italia,False,0,0,NaN,2021-03-06 19:30:32
4,"Btw, lower-income countries are not currently ...",Pallavi Guniganti,13014,1318,,False,0,0,NaN,2021-03-06 19:30:18


In [33]:
df.shape

(1000, 10)

In [34]:
from textblob import TextBlob

In [35]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z\t])|(\w+:\/\/\S+)',' ',tweet).split())

In [36]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity==0:
        return 'Neutral'
    else:
        return 'Negative'

In [37]:
df['clean_tweets'] = df['Tweets'].apply(lambda x: clean_tweet(x))

In [38]:
df[['Tweets','clean_tweets']].head()

,Tweets,clean_tweets
0,RT @DalaiLama: His Holiness the Dalai Lama rec...,RT His Holiness the Dalai Lama receiving the f...
1,RT @DalaiLama: His Holiness the Dalai Lama rec...,RT His Holiness the Dalai Lama receiving the f...
2,RT @AabidMagami: Breaking :\n\nGujarat Man Inf...,RT Breaking Gujarat Man Infected With Covid Af...
3,#Bollywood #HemaMalini #vaccination #SaifAliKh...,Bollywood HemaMalini vaccination SaifAliKhan K...
4,"Btw, lower-income countries are not currently ...",Btw lower income countries are not currently p...


In [39]:
df['Sentiment'] = df['clean_tweets'].apply(lambda x: analyze_sentiment(x))

In [40]:
df[['clean_tweets','Sentiment']].head()

,clean_tweets,Sentiment
0,RT His Holiness the Dalai Lama receiving the f...,Positive
1,RT His Holiness the Dalai Lama receiving the f...,Positive
2,RT Breaking Gujarat Man Infected With Covid Af...,Neutral
3,Bollywood HemaMalini vaccination SaifAliKhan K...,Neutral
4,Btw lower income countries are not currently p...,Neutral


In [41]:
n = 20
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweets'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @iamhimanshu_raj: Finally India among the very few countries which have R&amp;D and produced it's own Covid-19 vaccine 🤘🏻🤘🏻🤘🏻

I don't know…

Clean Tweet:
RT raj Finally India among the very few countries which have R amp D and produced it s own Covid 19 vaccine I don t know

Sentiment:
Positive


In [42]:
n = 50
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweets'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @MajorPoonia: Finally Pakistan requests for made in India Covid Vaccine under Global Alliance for Vaccines and Immunisation program.
@fa…

Clean Tweet:
RT Finally Pakistan requests for made in India Covid Vaccine under Global Alliance for Vaccines and Immunisation program

Sentiment:
Neutral


In [43]:
df[df.Sentiment=='Positive'].shape[0]

374

In [44]:
df[df.Sentiment=='Neutral'].shape[0]

592

In [45]:
df[df.Sentiment=='Negative'].shape[0]

34